In [1]:
# [Celda 1] - Importación de librerías
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
import optuna
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import VotingClassifier

In [2]:
# [Celda 2] - Carga de datos
# Leemos el dataset
df = pd.read_csv('../data/cleaned_data.csv')

# Mostramos las primeras filas y la información del dataset
print("Primeras filas del dataset:")
display(df.head())
print("\nInformación del dataset:")
display(df.info())

Primeras filas del dataset:


,Cleaned_Text,Any_Hate,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsSexist,IsHomophobic,IsReligiousHate,IsRadicalism
0,people would take step back make case anyone e...,False,False,False,False,False,False,False,False,False,False,False,False,False
1,law enforcement trained shoot apprehend traine...,True,True,True,False,False,False,False,False,False,False,False,False,False
2,dont reckon life matter banner held white cunt...,True,True,True,False,False,True,False,False,False,False,False,False,False
3,large number people like police officer called...,False,False,False,False,False,False,False,False,False,False,False,False,False
4,arab dude absolutely right shot 6 extra time s...,False,False,False,False,False,False,False,False,False,False,False,False,False



Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Cleaned_Text     1000 non-null   object
 1   Any_Hate         1000 non-null   bool  
 2   IsToxic          1000 non-null   bool  
 3   IsAbusive        1000 non-null   bool  
 4   IsThreat         1000 non-null   bool  
 5   IsProvocative    1000 non-null   bool  
 6   IsObscene        1000 non-null   bool  
 7   IsHatespeech     1000 non-null   bool  
 8   IsRacist         1000 non-null   bool  
 9   IsNationalist    1000 non-null   bool  
 10  IsSexist         1000 non-null   bool  
 11  IsHomophobic     1000 non-null   bool  
 12  IsReligiousHate  1000 non-null   bool  
 13  IsRadicalism     1000 non-null   bool  
dtypes: bool(13), object(1)
memory usage: 20.6+ KB


None

In [3]:
# [Celda 3] - Preparación y limpieza de datos

# Primero mostramos información sobre valores nulos en el dataset
print("Valores nulos por columna antes de la limpieza:")
print(df.isnull().sum())
print("\nTotal de filas antes de la limpieza:", len(df))

# Definimos las columnas que indican contenido dañino
toxic_columns = ['IsToxic', 'IsAbusive', 'IsThreat', 'IsProvocative', 
                'IsObscene', 'IsHatespeech', 'IsRacist', 'IsNationalist',
                'IsSexist', 'IsHomophobic', 'IsReligiousHate', 'IsRadicalism']

# Convertir los valores "TRUE"/"FALSE" a 1/0 en las columnas tóxicas (si es necesario)
df[toxic_columns] = df[toxic_columns].replace({'TRUE': 1, 'FALSE': 0})

# Creamos una nueva columna que será 1 si cualquiera de las categorías es 1
df['is_harmful'] = df[toxic_columns].any(axis=1).astype(int)

# Preparamos X (textos) e y (etiquetas)
X = df['Cleaned_Text']
y = df['is_harmful']

# Mostramos estadísticas finales
print("\nEstadísticas finales:")
print("-" * 50)
print("Distribución de mensajes dañinos vs no dañinos:")
print(y.value_counts(normalize=True))
print("\nDistribución por tipo de contenido dañino:")
for col in toxic_columns:
    positivos = df[col].sum()
    porcentaje = (positivos / len(df)) * 100
    print(f"{col}: {positivos} casos ({porcentaje:.2f}%)")


Valores nulos por columna antes de la limpieza:
Cleaned_Text       0
Any_Hate           0
IsToxic            0
IsAbusive          0
IsThreat           0
IsProvocative      0
IsObscene          0
IsHatespeech       0
IsRacist           0
IsNationalist      0
IsSexist           0
IsHomophobic       0
IsReligiousHate    0
IsRadicalism       0
dtype: int64

Total de filas antes de la limpieza: 1000

Estadísticas finales:
--------------------------------------------------
Distribución de mensajes dañinos vs no dañinos:
is_harmful
0    0.538
1    0.462
Name: proportion, dtype: float64

Distribución por tipo de contenido dañino:
IsToxic: 462 casos (46.20%)
IsAbusive: 353 casos (35.30%)
IsThreat: 21 casos (2.10%)
IsProvocative: 161 casos (16.10%)
IsObscene: 100 casos (10.00%)
IsHatespeech: 138 casos (13.80%)
IsRacist: 125 casos (12.50%)
IsNationalist: 8 casos (0.80%)
IsSexist: 1 casos (0.10%)
IsHomophobic: 0 casos (0.00%)
IsReligiousHate: 12 casos (1.20%)
IsRadicalism: 0 casos (0.00%)


In [4]:
# [Celda 4] - División de datos
# Dividimos los datos en conjuntos de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.15,
    random_state=42,
    stratify=y  # Mantenemos la proporción de clases
)

print("Tamaño del conjunto de entrenamiento:", len(x_train))
print("Tamaño del conjunto de prueba:", len(x_test))

Tamaño del conjunto de entrenamiento: 850
Tamaño del conjunto de prueba: 150


In [5]:
# [Celda 5] - Vectorización del texto
# Convertimos el texto a una matriz de características TF-IDF
vectorizer = TfidfVectorizer(
    max_features=5000,  # Limitamos el número de características
    min_df=2,          # Ignoramos términos que aparecen en menos de 2 documentos
    stop_words='english'  # Removemos stopwords en inglés
)

# Transformamos los textos
x_train_vectorized = vectorizer.fit_transform(x_train)
x_test_vectorized = vectorizer.transform(x_test)

print("Dimensiones de la matriz de características de entrenamiento:", x_train_vectorized.shape)

Dimensiones de la matriz de características de entrenamiento: (850, 1421)


In [6]:
# Extraer los mejores parámetros

# Crear el modelo SVM con los mejores hiperparámetros hardcodeados
final_svm_model = SVC(
    C=109.42844096035671,
    kernel='rbf',
    gamma=1.1662316858478285,
    probability=True,
    random_state=42,
    class_weight=None,
    tol=0.00043366566542372227,
    shrinking=False,
    max_iter=1697
)

# Entrenar el modelo con el conjunto de entrenamiento completo
final_svm_model.fit(x_train_vectorized, y_train)

print("Modelo final entrenado con los mejores hiperparámetros!")



Modelo final entrenado con los mejores hiperparámetros!


In [7]:
# Predicciones de probabilidades para las clases
y_prob = final_svm_model.predict_proba(x_test_vectorized)[:, 1]  # Probabilidad de la clase positiva (por ejemplo, clase 1)

# Ajuste del umbral de decisión
threshold = 0.46  # Cambia este valor según lo que desees probar
y_pred = (y_prob >= threshold).astype(int)  # Predicciones basadas en el umbral

# Métricas de rendimiento
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy en conjunto de prueba (con umbral {threshold}):", accuracy)
print("Reporte de clasificación:\n", report)

Accuracy en conjunto de prueba (con umbral 0.46): 0.7266666666666667
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.73      0.78      0.75        81
           1       0.72      0.67      0.69        69

    accuracy                           0.73       150
   macro avg       0.73      0.72      0.72       150
weighted avg       0.73      0.73      0.73       150



In [8]:
import optuna
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

# Definir la función objetivo para Optuna con MultinomialNB
def objective_nb(trial):
    # Definir los hiperparámetros a optimizar
    alpha = trial.suggest_loguniform('alpha', 1e-5, 1e2)  # Parámetro de suavizado
    fit_prior = trial.suggest_categorical('fit_prior', [True, False])  # Ajustar o no las probabilidades previas
    
    # Crear el modelo Naive Bayes con los hiperparámetros seleccionados
    nb_model = MultinomialNB(alpha=alpha, fit_prior=fit_prior)
    
    # Evaluar el modelo con validación cruzada
    scores = cross_val_score(nb_model, x_train_vectorized, y_train, cv=3, scoring='accuracy')
    accuracy = scores.mean()  # Promedio de accuracy
    
    return accuracy  # Optuna maximiza este valor

# Crear un estudio de optimización para MultinomialNB
study_nb = optuna.create_study(direction='maximize')
study_nb.optimize(objective_nb, n_trials=50)  # Ajusta el número de pruebas según el tiempo disponible

# Resultados de la mejor prueba para MultinomialNB
print("Mejores hiperparámetros para MultinomialNB:")
print(study_nb.best_trial.params)
print("Mejor precisión:", study_nb.best_trial.value)

[I 2024-11-11 17:43:54,794] A new study created in memory with name: no-name-6e9c64d9-8267-49ee-8077-255bd7b3e344
C:\Users\maria\AppData\Local\Temp\ipykernel_440\1044462155.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform('alpha', 1e-5, 1e2)  # Parámetro de suavizado
[I 2024-11-11 17:43:54,810] Trial 0 finished with value: 0.6753139567676968 and parameters: {'alpha': 19.359799819449584, 'fit_prior': False}. Best is trial 0 with value: 0.6753139567676968.
C:\Users\maria\AppData\Local\Temp\ipykernel_440\1044462155.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform('alpha', 1e-5, 1e2)  # Parámetro de sua

Mejores hiperparámetros para MultinomialNB:
{'alpha': 1.0359637740543461, 'fit_prior': True}
Mejor precisión: 0.7070787504769488


In [9]:
# Extraer los mejores hiperparámetros del estudio de MultinomialNB
best_params_nb = study_nb.best_trial.params

# Crear el modelo final con los mejores hiperparámetros
final_nb_model = MultinomialNB(
    alpha=best_params_nb['alpha'],
    fit_prior=best_params_nb['fit_prior']
)

# Entrenar el modelo con el conjunto de entrenamiento completo
final_nb_model.fit(x_train_vectorized, y_train)

# Evaluar el modelo en el conjunto de prueba
y_pred_nb = final_nb_model.predict(x_test_vectorized)

# Calcular y mostrar la precisión y el reporte de clasificación
from sklearn.metrics import accuracy_score, classification_report
accuracy_nb = accuracy_score(y_test, y_pred_nb)
report_nb = classification_report(y_test, y_pred_nb)

print("Accuracy en conjunto de prueba para MultinomialNB:", accuracy_nb)
print("Reporte de clasificación para MultinomialNB:\n", report_nb)

Accuracy en conjunto de prueba para MultinomialNB: 0.6933333333333334
Reporte de clasificación para MultinomialNB:
               precision    recall  f1-score   support

           0       0.69      0.78      0.73        81
           1       0.69      0.59      0.64        69

    accuracy                           0.69       150
   macro avg       0.69      0.69      0.69       150
weighted avg       0.69      0.69      0.69       150



In [10]:
from sklearn.ensemble import RandomForestClassifier

# Definir la función objetivo para Optuna con RandomForestClassifier
def objective_rf(trial):
    # Definir los hiperparámetros a optimizar
    n_estimators = trial.suggest_int('n_estimators', 50, 300)  # Número de árboles
    max_depth = trial.suggest_int('max_depth', 5, 50)  # Profundidad máxima de cada árbol
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)  # Tamaño mínimo para hacer split
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)  # Tamaño mínimo de muestras en una hoja
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2'])  # Características a considerar
    
    # Crear el modelo Random Forest con los hiperparámetros seleccionados
    rf_model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=42
    )
    
    # Evaluar el modelo con validación cruzada
    scores = cross_val_score(rf_model, x_train_vectorized, y_train, cv=3, scoring='accuracy')
    accuracy = scores.mean()  # Promedio de accuracy
    
    return accuracy  # Optuna maximiza este valor

# Crear un estudio de optimización para RandomForestClassifier
study_rf = optuna.create_study(direction='maximize')
study_rf.optimize(objective_rf, n_trials=50)  # Ajusta el número de pruebas según el tiempo disponible

# Resultados de la mejor prueba para RandomForestClassifier
print("Mejores hiperparámetros para RandomForestClassifier:")
print(study_rf.best_trial.params)
print("Mejor precisión:", study_rf.best_trial.value)

[I 2024-11-11 17:43:55,471] A new study created in memory with name: no-name-9ceae4ae-6ac6-4cb6-ab2e-2a867f3f4094
[I 2024-11-11 17:43:56,011] Trial 0 finished with value: 0.5376457804542213 and parameters: {'n_estimators': 191, 'max_depth': 36, 'min_samples_split': 8, 'min_samples_leaf': 8, 'max_features': 'log2'}. Best is trial 0 with value: 0.5376457804542213.
[I 2024-11-11 17:43:56,863] Trial 1 finished with value: 0.6753222515303837 and parameters: {'n_estimators': 217, 'max_depth': 39, 'min_samples_split': 14, 'min_samples_leaf': 1, 'max_features': 'log2'}. Best is trial 1 with value: 0.6753222515303837.
[I 2024-11-11 17:43:57,162] Trial 2 finished with value: 0.588231390699912 and parameters: {'n_estimators': 99, 'max_depth': 44, 'min_samples_split': 5, 'min_samples_leaf': 9, 'max_features': 'sqrt'}. Best is trial 1 with value: 0.6753222515303837.
[I 2024-11-11 17:43:57,902] Trial 3 finished with value: 0.5517593191658786 and parameters: {'n_estimators': 242, 'max_depth': 6, 'min

Mejores hiperparámetros para RandomForestClassifier:
{'n_estimators': 196, 'max_depth': 42, 'min_samples_split': 9, 'min_samples_leaf': 2, 'max_features': 'sqrt'}
Mejor precisión: 0.6964904859071982


In [11]:
# Crear el modelo final con los mejores hiperparámetros
final_rf_model = RandomForestClassifier(
    n_estimators=235,
    max_depth=44,
    min_samples_split=10,
    min_samples_leaf=2,
    max_features='sqrt',
    random_state=42
)

# Entrenar el modelo con el conjunto de entrenamiento completo
final_rf_model.fit(x_train_vectorized, y_train)

# Evaluar el modelo en el conjunto de prueba
y_pred_rf = final_rf_model.predict(x_test_vectorized)

# Calcular y mostrar la precisión y el reporte de clasificación
accuracy_rf = accuracy_score(y_test, y_pred_rf)
report_rf = classification_report(y_test, y_pred_rf)

print("Accuracy en conjunto de prueba para RandomForestClassifier:", accuracy_rf)
print("Reporte de clasificación para RandomForestClassifier:\n", report_rf)


Accuracy en conjunto de prueba para RandomForestClassifier: 0.72
Reporte de clasificación para RandomForestClassifier:
               precision    recall  f1-score   support

           0       0.68      0.90      0.78        81
           1       0.81      0.51      0.62        69

    accuracy                           0.72       150
   macro avg       0.75      0.70      0.70       150
weighted avg       0.74      0.72      0.71       150



In [12]:
# Crear el ensemble con los modelos finales
ensemble_model = VotingClassifier(
    estimators=[
        ('random_forest', final_rf_model),
        ('naive_bayes', final_nb_model),
        ('svm', final_svm_model)
    ],
    voting='soft'  # Votación suave para promediar probabilidades
)

# Entrenar el ensemble con el conjunto de entrenamiento
ensemble_model.fit(x_train_vectorized, y_train)

# Evaluar el modelo en el conjunto de prueba
y_pred_ensemble = ensemble_model.predict(x_test_vectorized)

# Calcular y mostrar la precisión y el reporte de clasificación
accuracy_ensemble = accuracy_score(y_test, y_pred_ensemble)
report_ensemble = classification_report(y_test, y_pred_ensemble)

print("Accuracy en conjunto de prueba para el ensemble:", accuracy_ensemble)
print("Reporte de clasificación para el ensemble:\n", report_ensemble)

Accuracy en conjunto de prueba para el ensemble: 0.7333333333333333
Reporte de clasificación para el ensemble:
               precision    recall  f1-score   support

           0       0.71      0.85      0.78        81
           1       0.77      0.59      0.67        69

    accuracy                           0.73       150
   macro avg       0.74      0.72      0.72       150
weighted avg       0.74      0.73      0.73       150



In [13]:
# Evaluar el modelo en el conjunto de entrenamiento utilizando el nuevo threshold
y_prob_train_ensemble = ensemble_model.predict_proba(x_train_vectorized)[:, 1]  # Probabilidades de la clase positiva

# Definir un nuevo threshold
threshold = 0.759 # Puedes cambiar este valor según lo que necesites

# Aplicar el threshold para obtener las predicciones
y_pred_train_ensemble_thresholded = (y_prob_train_ensemble >= threshold).astype(int)

# Calcular la precisión en el conjunto de entrenamiento con el nuevo threshold
accuracy_train_ensemble = accuracy_score(y_train, y_pred_train_ensemble_thresholded)

# Evaluar en el conjunto de prueba con el nuevo threshold
y_prob_test_ensemble = ensemble_model.predict_proba(x_test_vectorized)[:, 1]
y_pred_test_ensemble_thresholded = (y_prob_test_ensemble >= threshold).astype(int)

# Calcular la precisión en el conjunto de prueba con el nuevo threshold
accuracy_test_ensemble = accuracy_score(y_test, y_pred_test_ensemble_thresholded)

# Calcular el porcentaje de overfitting
overfitting_percentage = ((accuracy_train_ensemble - accuracy_test_ensemble) / accuracy_train_ensemble) * 100

# Imprimir resultados
print("Accuracy en conjunto de entrenamiento para el ensemble con threshold de", threshold, ":", accuracy_train_ensemble)
print("Accuracy en conjunto de prueba para el ensemble con threshold de", threshold, ":", accuracy_test_ensemble)
print("Porcentaje de overfitting del ensemble:", overfitting_percentage, "%")


Accuracy en conjunto de entrenamiento para el ensemble con threshold de 0.759 : 0.7658823529411765
Accuracy en conjunto de prueba para el ensemble con threshold de 0.759 : 0.5866666666666667
Porcentaje de overfitting del ensemble: 23.399897593445978 %


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
import numpy as np
from scipy import sparse

# 1. Primero dividimos los datos en conjuntos de entrenamiento y prueba
x_temp, x_test, y_temp, y_test = train_test_split(
    X, y, 
    test_size=0.01,
    random_state=42,
    stratify=y
)

# Separamos el conjunto temporal en entrenamiento y validación
x_train, x_val, y_train, y_val = train_test_split(
    x_temp, y_temp,
    test_size=0.1,
    random_state=42,
    stratify=y_temp
)

# 2. Vectorizamos los datos
vectorizer = TfidfVectorizer(
    min_df=2,
    max_df=0.95,
    max_features=5000
)

# Vectorizamos cada conjunto por separado
x_train_vectorized = vectorizer.fit_transform(x_train)
x_val_vectorized = vectorizer.transform(x_val)
x_test_vectorized = vectorizer.transform(x_test)

# 3. Aplicamos la reducción de dimensionalidad
selector = SelectKBest(score_func=chi2, k=1000)
x_train_reduced = selector.fit_transform(x_train_vectorized, y_train)
x_val_reduced = selector.transform(x_val_vectorized)
x_test_reduced = selector.transform(x_test_vectorized)

print("Dimensiones después de la reducción:")
print("Entrenamiento:", x_train_reduced.shape)
print("Validación:", x_val_reduced.shape)
print("Prueba:", x_test_reduced.shape)

# 4. Creamos y entrenamos el ensemble con bagging
bagged_ensemble = BaggingClassifier(
    estimator=VotingClassifier(
        estimators=[
            ('random_forest', final_rf_model),
            ('naive_bayes', final_nb_model),
            ('svm', final_svm_model)
        ],
        voting='soft'
    ),
    n_estimators=10,
    max_samples=0.8,
    max_features=0.8,
    random_state=42
)

# 5. Implementamos early stopping
class EarlyStoppingMonitor:
    def __init__(self, patience=3, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.best_score = None
        self.counter = 0
        self.best_model = None
        
    def check(self, model, val_score):
        if self.best_score is None or val_score > self.best_score + self.min_delta:
            self.best_score = val_score
            self.counter = 0
            self.best_model = model
            return False
        else:
            self.counter += 1
            return self.counter >= self.patience

# Entrenamiento con early stopping
monitor = EarlyStoppingMonitor(patience=3)
best_accuracy = 0

for epoch in range(10):
    # Entrenar el modelo
    bagged_ensemble.fit(x_train_reduced, y_train)
    
    # Evaluar en el conjunto de validación
    val_accuracy = bagged_ensemble.score(x_val_reduced, y_val)
    print(f"Época {epoch + 1}, Accuracy en validación: {val_accuracy:.4f}")
    
    if monitor.check(bagged_ensemble, val_accuracy):
        print("Early stopping activado en época", epoch + 1)
        break

# Usar el mejor modelo encontrado
best_model = monitor.best_model if monitor.best_model is not None else bagged_ensemble


Dimensiones después de la reducción:
Entrenamiento: (891, 1000)
Validación: (99, 1000)
Prueba: (10, 1000)
Época 1, Accuracy en validación: 0.7677
Época 2, Accuracy en validación: 0.7677
Época 3, Accuracy en validación: 0.7677
Época 4, Accuracy en validación: 0.7677
Early stopping activado en época 4


In [15]:
# Evaluar el mejor modelo con threshold
threshold = 0.59

# Predicciones en entrenamiento
y_prob_train = best_model.predict_proba(x_train_reduced)[:, 1]
y_pred_train_thresholded = (y_prob_train >= threshold).astype(int)

# Predicciones en prueba
y_prob_test = best_model.predict_proba(x_test_reduced)[:, 1]
y_pred_test_thresholded = (y_prob_test >= threshold).astype(int)

# Calcular métricas
accuracy_train = accuracy_score(y_train, y_pred_train_thresholded)
accuracy_test = accuracy_score(y_test, y_pred_test_thresholded)
overfitting_percentage = ((accuracy_train - accuracy_test) / accuracy_train) * 100

# Imprimir resultados
print("\nResultados finales:")
print(f"Accuracy en entrenamiento (threshold={threshold}): {accuracy_train:.4f}")
print(f"Accuracy en prueba (threshold={threshold}): {accuracy_test:.4f}")
print(f"Porcentaje de overfitting: {overfitting_percentage:.2f}%")
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred_test_thresholded))

# Visualizar las características más importantes
if hasattr(selector, 'get_feature_names_out'):
    feature_names = vectorizer.get_feature_names_out()
    selected_features = selector.get_support()
    important_features = [(feature_names[i], selector.scores_[i]) 
                         for i in range(len(feature_names)) 
                         if selected_features[i]]
    
    important_features.sort(key=lambda x: x[1], reverse=True)
    print("\nTop 10 características más importantes:")
    for feature, score in important_features[:10]:
        print(f"{feature}: {score:.4f}")


Resultados finales:
Accuracy en entrenamiento (threshold=0.59): 0.8204
Accuracy en prueba (threshold=0.59): 0.6000
Porcentaje de overfitting: 26.87%

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.57      0.80      0.67         5
           1       0.67      0.40      0.50         5

    accuracy                           0.60        10
   macro avg       0.62      0.60      0.58        10
weighted avg       0.62      0.60      0.58        10


Top 10 características más importantes:
run: 11.2102
fuck: 9.5244
idiot: 8.7847
shit: 6.7682
as: 5.8536
bitch: 5.6284
fucking: 5.5250
dumb: 5.3711
peggy: 5.1694
thug: 4.6905


In [16]:
import joblib

# Crear un diccionario con todos los componentes
model_components = {
    'vectorizer': vectorizer,
    'selector': selector,
    'model': best_model,
    'threshold': threshold
}

# Guardar el modelo
joblib.dump(model_components, '../models/ensemble_model_complete.joblib')

print("Modelo guardado exitosamente!")

# Para cargar y usar el modelo más tarde:
def load_and_predict(text):
    # Cargar el modelo
    loaded_components = joblib.load('../models/ensemble_model_complete.joblib')
    
    # Extraer componentes
    vectorizer = loaded_components['vectorizer']
    selector = loaded_components['selector']
    model = loaded_components['model']
    threshold = loaded_components['threshold']
    
    # Procesar el texto nuevo
    x_new_vectorized = vectorizer.transform([text])
    x_new_reduced = selector.transform(x_new_vectorized)
    
    # Predecir
    probability = model.predict_proba(x_new_reduced)[0, 1]
    prediction = 1 if probability >= threshold else 0
    
    return prediction, probability

# # Ejemplo de uso:

# # Cargar y usar el modelo
# texto_ejemplo = "Ejemplo de texto para clasificar..."
# prediccion, probabilidad = load_and_predict(texto_ejemplo)
# print(f"Predicción: {prediccion}")
# print(f"Probabilidad: {probabilidad:.4f}")


Modelo guardado exitosamente!
